In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Load Dataset

In [6]:
from torchvision import transforms as T
from hcmus.core import pconf
from hcmus.utils.viz_utils import draw_boxes
from hcmus.lbs import LabelStudioConnector
from hcmus.data import LbsDataset

In [7]:
product_connector = LabelStudioConnector(
    url=pconf.LABEL_STUDIO_URL,
    api_key=pconf.LABEL_STUDIO_API_KEY,
    project_id=pconf.LABEL_STUDIO_PROJECT_ID,
)

In [9]:
from torchvision import transforms as T
# transforms = T.Compose([
#     T.Resize((800, 800)),  # Resize
# ])
product_dataset = LbsDataset(product_connector)

Loading tasks:   0%|          | 0/100 [00:00<?, ?it/s]

Loading tasks:  16%|█▌        | 16/100 [00:06<00:34,  2.42it/s]
2025-03-18 21:19:57.935 | INFO     | hcmus.data._lbs_dataset:__init__:25 - Number of labels: 203
2025-03-18 21:19:57.937 | INFO     | hcmus.data._lbs_dataset:__init__:26 - Number of data points: 1591


In [5]:
tensor, target = product_dataset[3]
image = product_dataset.tensor_to_image(tensor)

In [9]:
product_dataloader = product_dataset.get_dataloader(shuffle=False, batch_size=4)

# 2. Finetune Model

In [10]:
import torchvision
import torch.optim as optim
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
device = "mps"

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
num_classes = len(product_dataset._labels)
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.0005, momentum=0.9, weight_decay=0.0005)

In [ ]:
model.to(device)
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, targets in product_dataloader:
        optimizer.zero_grad()
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())

        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        print(f"Loss: {loss:.4f}")

print("Training Complete!")